<img src='https://www.icos-cp.eu/sites/default/files/2017-11/ICOS_CP_logo.png' width=400 align=right>

# ICOS Carbon Portal Python Library
## Example: STILT: footprints and timeseries

This example shows how to load timeseries data and footprints, and make some plots using Holoviews and Geoviews to create a map.

## Documentation
Full documentation for the library on the [project page](https://icos-carbon-portal.github.io/pylib/), how to install and wheel on [pypi.org](https://pypi.org/project/icoscp/"), source is available on [github](https://github.com/ICOS-Carbon-Portal/pylib)

In [1]:
# import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib widget

import pandas as pd

#Import STILT tools:
from icoscp.stilt import stiltstation

### Create a STILT station object

In [2]:
st = stiltstation.get(id='kit100')
print(st)

  0%|          | 0/170 [00:00<?, ?it/s]

{"id:": "KIT100", "name:": "KIT Karlsruhe 100m", "lat:": 49.09, "lon:": 8.43, "alt [m]:": 100, "country": {"common": "Germany", "official": "Federal Republic of Germany", "native": {"deu": {"official": "Bundesrepublik Deutschland", "common": "Deutschland"}}}}


### Get time series data

In [3]:
start = '2018-01-01'
end = '2018-12-31'

data = st.get_ts(start, end)
data.head()

,isodate,co2.stilt,co2.fuel,co2.bio,co2.background
date,,,,,
2018-01-01 00:00:00,1.514765e+09,413.143475,3.221731,2.403763,407.349177
2018-01-01 03:00:00,1.514776e+09,410.528320,1.637321,1.537977,407.294744
2018-01-01 06:00:00,1.514786e+09,410.118267,1.656064,1.483178,406.948165
2018-01-01 09:00:00,1.514797e+09,411.312663,2.844276,1.411304,407.015383
2018-01-01 12:00:00,1.514808e+09,410.124651,1.810577,1.090457,407.017072


### Plot STILT time series

In [4]:
data.plot(y=['co2.stilt', 'co2.background'], title=st.id, ylabel='ppm', figsize=(8,4))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:title={'center':'KIT100'}, xlabel='date', ylabel='ppm'>

## Extract time series with columns = 'co2'
see documentation what columns you can return
[https://icos-carbon-portal.github.io/pylib/modules/#get_tsstart_date-end_date-hours-columns](https://icos-carbon-portal.github.io/pylib/modules/#get_tsstart_date-end_date-hours-columns)

In [5]:
# set date constraints for the rest of this example notebook
start = '2018-01-01'
end = '2018-01-31'

In [6]:
stiltdata = st.get_ts(start, end, columns='co2')
stiltdata.head()

,isodate,co2.stilt,co2.fuel,co2.bio,co2.fuel.coal,co2.fuel.oil,co2.fuel.gas,co2.fuel.bio,co2.energy,co2.transport,co2.industry,co2.others,co2.cement,co2.background
date,,,,,,,,,,,,,,
2018-01-01 00:00:00,1.514765e+09,413.143475,3.221731,2.403763,0.286773,1.343621,1.119607,0.420045,1.271024,0.628513,0.430774,1.060224,0.168803,407.349177
2018-01-01 03:00:00,1.514776e+09,410.528320,1.637321,1.537977,0.264017,0.544036,0.619384,0.192084,1.034464,0.145997,0.218527,0.296610,0.058278,407.294744
2018-01-01 06:00:00,1.514786e+09,410.118267,1.656064,1.483178,0.279643,0.527059,0.631566,0.198606,1.071911,0.113382,0.109647,0.391984,0.030859,406.948165
2018-01-01 09:00:00,1.514797e+09,411.312663,2.844276,1.411304,0.381485,1.080131,1.007604,0.345954,1.281742,0.414996,0.171809,1.017430,0.041701,407.015383
2018-01-01 12:00:00,1.514808e+09,410.124651,1.810577,1.090457,0.266500,0.680725,0.587117,0.256929,0.882811,0.230719,0.289990,0.613603,0.206545,407.017072


In [7]:
stiltdata.columns

Index(['isodate', 'co2.stilt', 'co2.fuel', 'co2.bio', 'co2.fuel.coal',
       'co2.fuel.oil', 'co2.fuel.gas', 'co2.fuel.bio', 'co2.energy',
       'co2.transport', 'co2.industry', 'co2.others', 'co2.cement',
       'co2.background'],
      dtype='object')

In [8]:
stiltdata.columns[2:13].to_list()

['co2.fuel',
 'co2.bio',
 'co2.fuel.coal',
 'co2.fuel.oil',
 'co2.fuel.gas',
 'co2.fuel.bio',
 'co2.energy',
 'co2.transport',
 'co2.industry',
 'co2.others',
 'co2.cement']

### Create a plot with all co2 components

In [9]:
ax = stiltdata[stiltdata.columns[2:13].to_list()].plot()
ax.legend(loc='best', fontsize=8)
ax.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]

### Aggregate by day
and create a stacked bar graph

In [10]:
day = stiltdata.resample('D').sum()

In [11]:
# plot the bar graph
ax1 = day[stiltdata.columns[2:13].to_list()].plot.bar(stacked='True')
ax1.legend(loc='best', fontsize=8)

# adjust the xticks
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

# display
ax1.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]

## Load observation and compare to model result

In [12]:
from icoscp.cpb.dobj import Dobj

In [13]:
kit100 = Dobj('https://hdl.handle.net/11676/LJ4uetvEho7-k9K9TUnLHfFh')

In [14]:
kit100.citation

'Kubistin, D., Plaß-Dülmer, C., Arnold, S., Lindauer, M., Müller-Williams, J., Schumacher, M., ICOS RI, 2021. ICOS ATC CO2 Release, Karlsruhe (100.0 m), 2016-12-16–2021-01-31, https://hdl.handle.net/11676/LJ4uetvEho7-k9K9TUnLHfFh'

### create a mask to get the same timeframe

In [15]:
# we have set start and end date above (cell number 5)
mask = (kit100.data['TIMESTAMP'] >= start) & (kit100.data['TIMESTAMP'] <= end)
obsdata = kit100.data[mask]
obsdata.set_index('TIMESTAMP', inplace=True)
obsdata['co2']

TIMESTAMP
2018-01-01 00:00:00    412.360992
2018-01-01 01:00:00    412.373993
2018-01-01 02:00:00    413.381989
2018-01-01 03:00:00    416.872009
2018-01-01 04:00:00    414.395996
                          ...    
2018-01-30 20:00:00    430.531006
2018-01-30 21:00:00    428.112000
2018-01-30 22:00:00    431.802002
2018-01-30 23:00:00    434.117004
2018-01-31 00:00:00    431.378998
Name: co2, Length: 721, dtype: float64

### Resample STILT data
Because the observation are hourly aggregates, we resample the STILT output to make our lives easier to compare the observation vs model.

In [16]:
stilthourly = stiltdata.resample('1H').mean().interpolate('linear')
stilthourly['co2.stilt']

date
2018-01-01 00:00:00    413.143475
2018-01-01 01:00:00    412.271756
2018-01-01 02:00:00    411.400038
2018-01-01 03:00:00    410.528320
2018-01-01 04:00:00    410.391636
                          ...    
2018-01-31 17:00:00    416.111737
2018-01-31 18:00:00    415.513248
2018-01-31 19:00:00    415.647334
2018-01-31 20:00:00    415.781419
2018-01-31 21:00:00    415.915505
Freq: H, Name: co2.stilt, Length: 742, dtype: float64

### Data harmonization and plot
If you look at the lenght of the dataframes above you will see a discrepancy. Observation contains less data points. Most likely an interuption of the measurement or QA/QC removed values. We need to merge the files together on index (both have now the Time/Date se as index). Missing values will be NaN in the pandas data frame

In [17]:
harmonized = stilthourly.join(obsdata)
harmonized.plot(y = ['co2.stilt', 'co2'], use_index=True, grid=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='date'>

### Plot difference

In [18]:
harmonized['diff'] = harmonized['co2.stilt']-harmonized['co2']

In [19]:
plt.figure(figsize=(8,4))
#plt.grid(color='0.9')
ax2 = plt.axes()
ax2.plot(harmonized['diff'])
ax2.grid(color='0.9')

# adjust the xticks
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …